In [ ]:
import tensorflow as tf
import numpy as np
from collections import deque
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from DataGenerator import DataGenerator

data_file = "sample_datasets/small_sequences.fasta"
processed_output_directory = "full_datasets/preprocessed/small_sequences/"
codon_input_length = 5 # number of codons to input to the model

: 

In [44]:
def stringToMultiDimArray(st):
    helper_stack = deque()
    for ch in st:
        if ch == '[':
            helper_stack.append([])
        elif ch == ']':
            current = helper_stack.pop()
            if len(helper_stack) > 0:
                helper_stack[-1].append(current)
            else:
                helper_stack.append(current)
        elif ch not in [' ', ',']:
            helper_stack[-1].append(int(ch))
    return helper_stack[0]


def parseSample(directory, id):
    sample_x_file_st = directory + str(id) + "_x.txt"
    sample_y_file_st = directory + str(id) + "_y.txt"
    sample_x_file = open(sample_x_file_st, 'r')
    sample_y_file = open(sample_y_file_st, 'r')
    sample_x_st = sample_x_file.read()
    sample_y_st = sample_y_file.read()
    sample_x = stringToMultiDimArray(sample_x_st)
    sample_y = stringToMultiDimArray(sample_y_st)
    print(np.array([sample_x]))
    return (sample_x, sample_y)

def epochGenerator(directory):
    def helper():
        files = os.listdir(directory)
        samples = len(files) // 2
        for i in range(samples):
            (x, y) = parseSample(directory, i)
            return (np.array([x]), np.array([y]))
            # return (np.array([x]), np.array([]))
    return helper()

In [52]:
# gpus = tf.config.list_logical_devices('GPU')
# strategy = tf.distribute.MirroredStrategy(gpus)
# with strategy.scope():
with tf.device('/GPU:1'):
  tf.debugging.set_log_device_placement(True)
  model = Sequential()
  model.add(LSTM(10, input_shape=(5, 64), return_sequences=True))
  model.add(LSTM(5, return_sequences=False))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='softmax'))
  model.summary()
  # model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(DataGenerator(processed_output_directory))
  # inputs = tf.keras.layers.Input(shape=(1,))
  # predictions = tf.keras.layers.Dense(1)(inputs)
  # model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
  # model.compile(loss='mse',
  #               optimizer=tf.keras.optimizers.SGD(learning_rate=0.2))

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_36 (LSTM)              (None, 5, 10)             3000      
                                                                 
 lstm_37 (LSTM)              (None, 5)                 320       
                                                                 
 dense_36 (Dense)            (None, 128)               768       
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
Total params: 12,344
Trainable params: 12,344
Non-trainable params: 0
_________________________________________________________________


2023-05-22 01:29:39.726229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-22 01:29:40.515585: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-22 01:29:40.515712: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 1, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-22 01:29:40.515789: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 11730 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2023-05-22 01:29:40.515821: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

InvalidArgumentError: Graph execution error:

No OpKernel was registered to support Op 'CudnnRNN' used by {{node CudnnRNN}} with these attrs: [direction="unidirectional", rnn_mode="lstm", seed2=0, is_training=true, seed=0, dropout=0, T=DT_FLOAT, input_mode="linear_input"]
Registered devices: [CPU, GPU]
Registered kernels:
  <no registered kernels>

	 [[CudnnRNN]]
	 [[sequential_18/lstm_36/PartitionedCall]] [Op:__inference_train_function_73505]